# Brazilian E-commerce Analysis


### Importing Libraries & Data Loading

In [1]:
import pandas as pd

import numpy as np

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import altair as alt

import itables


In [2]:
# Loading the raw CSV files into Pandas DataFrames
orders = pd.read_csv(r'D:\Full Projects\project4\2\olist_orders_dataset.csv')
items = pd.read_csv(r'D:\Full Projects\project4\2\olist_order_items_dataset.csv')
products = pd.read_csv(r'D:\Full Projects\project4\2\olist_products_dataset.csv')
payments = pd.read_csv(r'D:\Full Projects\project4\2\olist_order_payments_dataset.csv')
reviews = pd.read_csv(r'D:\Full Projects\project4\2\olist_order_reviews_dataset.csv')
customers = pd.read_csv(r'D:\Full Projects\project4\2\olist_customers_dataset.csv')
translation = pd.read_csv(r'D:\Full Projects\project4\2\product_category_name_translation.csv')
geolocation = pd.read_csv(r'D:\Full Projects\project4\2\olist_geolocation_dataset.csv')

print("All datasets loaded successfully.")


All datasets loaded successfully.


### Initial Cleaning 

In [3]:
date_cols = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date', 
             'order_estimated_delivery_date']

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# 2.2 Filter for 'Delivered' Orders Only
# We only want to analyze completed transactions for sales and delivery performance.
initial_count = len(orders)
orders_clean = orders[orders['order_status'] == 'delivered'].copy()
dropped_count = initial_count - len(orders_clean)

# 2.3 Handle Missing Delivery Dates
# If an order is marked 'delivered' but has no delivery date, it's data corruption.
orders_clean = orders_clean.dropna(subset=['order_delivered_customer_date'])

print(f"Orders cleaned. Dropped {dropped_count} non-delivered/canceled orders.")

#Translate Category Names to English
products = products.merge(translation, on='product_category_name', how='left')

# Handle Missing Translations
# Fill NaN values in the English column with 'Unknown' to avoid errors later
products['product_category_name_english'] = products['product_category_name_english'].fillna('Unknown')

# Drop the original Portuguese column to save memory
products.drop(columns=['product_category_name'], inplace=True)

print("Products translated and cleaned.")


#Handle Missing Review Messages
reviews['review_comment_message'] = reviews['review_comment_message'].fillna('')
reviews['review_comment_title'] = reviews['review_comment_title'].fillna('')

print("Reviews cleaned (NaNs replaced with empty strings).")

# Check for Duplicates
duplicates_orders = orders_clean.duplicated().sum()
duplicates_products = products.duplicated().sum()
duplicates_reviews = reviews.duplicated().sum()
duplicates_payments = payments.duplicated().sum()
duplicates_items = items.duplicated().sum()
duplicates_customers = customers.duplicated().sum()
duplicates_geolocation = geolocation.duplicated().sum()

duplicates_total = (duplicates_orders+ duplicates_products + 
                       duplicates_reviews  + duplicates_payments 
                    + duplicates_items + duplicates_customers+ duplicates_geolocation)
if duplicates_total > 0:
    orders_clean.drop_duplicates(inplace=True)
    products.drop_duplicates(inplace=True)
    reviews.drop_duplicates(inplace=True)
    payments.drop_duplicates(inplace=True)
    items.drop_duplicates(inplace=True)
    customers.drop_duplicates(inplace=True)
    geolocation.drop_duplicates(inplace=True)

print(f"Order Items cleaned. Removed {duplicates_total} duplicates.")

# ==============================================================================
# FINAL STATUS CHECK
# ==============================================================================
print(f"Clean Orders Count: {len(orders_clean)}")
print(f"Products Count: {len(products)}")
print(f"Reviews Count: {len(reviews)}")

Orders cleaned. Dropped 2963 non-delivered/canceled orders.
Products translated and cleaned.
Reviews cleaned (NaNs replaced with empty strings).
Order Items cleaned. Removed 261831 duplicates.
Clean Orders Count: 96470
Products Count: 32951
Reviews Count: 99224


### KPI CARDS CALCULATION

In [20]:
# ==============================================================================
# Calculating values for Dashboard Cards
# ==============================================================================

total_orders = orders_clean['order_id'].nunique()
avg_review_score = reviews['review_score'].mean()
total_customers = customers['customer_unique_id'].nunique()
on_time_orders = orders_clean[orders_clean['order_delivered_customer_date'] <= orders_clean['order_estimated_delivery_date']]
on_time_rate = (len(on_time_orders) / len(orders_clean)) * 100



# PRINTING THE CARDS

print(f"👥 Total Customers:    {total_customers:,}")
print(f"📦 Total Orders:       {total_orders:,}")
print(f"⭐ Avg Review Score:   {avg_review_score:.2f} / 5.0")
print(f"⏰ On-Time Rate:       {on_time_rate:.1f}%")

👥 Total Customers:    96,096
📦 Total Orders:       96,470
⭐ Avg Review Score:   4.09 / 5.0
⏰ On-Time Rate:       91.9%


### Top 10 Categories by Sales Volume

In [5]:
items_category = (items.merge(products, on='product_id'))
items_category = items_category.groupby('product_category_name_english')['price'].sum().reset_index(name='volume').sort_values(by='volume', ascending=False)
items_category.columns = ['Product Category', 'Total Sales Volume']

# Plotting Top 10 Product Categories by Sales Volume
top10 = items_category.head(10).copy()
total_volume = items_category['Total Sales Volume'].sum()
top10['Market Share (%)'] = (top10['Total Sales Volume'] / total_volume) * 100

# Create Horizontal Bar Chart with Market Share Annotations
fig = px.bar(
    top10,
    x="Total Sales Volume",
    y="Product Category",
    orientation="h",
    text=top10["Market Share (%)"].round(1).astype(str) + "%",
    title="Top 10 Product Categories by Sales Volume (with Market Share)",
    labels={"Total Sales Volume": "Sales Volume", "Product Category": "Category"}
)


# Reverse y-axis so the highest category is at the top
fig.update_layout(
    yaxis=dict(autorange="reversed"),
    template="plotly_white",
    title_font_size=20,
    xaxis_title="Total Sales Volume",
    yaxis_title="Product Category"
)

# Show the figure
fig.show()


#### Key Insights
- Market Concentration:
The top 3 categories — health_beauty, watches_gifts, and bed_bath_table — together account for over 25% of total sales volume, indicating a strong concentration in consumer demand.
- Category Leadership:
health_beauty leads the market with a 9.3% share, suggesting high demand and potential for targeted promotions or inventory expansion.
- Gift-Oriented Spending:
watches_gifts ranks second with 8.9%, reflecting seasonal or occasion-driven purchases that may benefit from calendar-based marketing strategies.
- Home & Lifestyle Focus:
Categories like bed_bath_table, furniture_decor, and housewares show consistent performance, indicating sustained interest in home improvement and lifestyle products.
- Tech & Leisure Balance:
computers_accessories and sports_leisure both appear in the top 5, suggesting a balanced mix of utility and recreation in consumer behavior.
- Long Tail Opportunity:
Lower-ranked categories like auto and garden_tools still hold notable market share (~3–4%), which may be optimized through niche targeting or bundling strategies.


### Payment Method Distribution

In [6]:
# ==============================================================================
# Question: How do customers prefer to pay?
# ==============================================================================


# Step 1: Data Cleaning
# Standardize payment type names (lowercase, replace underscores with spaces)
payments['payment_type'] = (
    payments['payment_type']
    .astype(str)
    .str.lower()
    .str.replace('_', ' ')
    .str.strip()
)

# Step 2: Calculate Distribution
# Count occurrences of each payment type
payment_counts = payments['payment_type'].value_counts().reset_index()
payment_counts.columns = ['payment_type', 'count']

# Calculate percentage for better interpretation
total_count = payment_counts['count'].sum()
payment_counts['percentage'] = (payment_counts['count'] / total_count) * 100

# Step 3: Remove invalid entries
# Filter out 'not defined' or unknown payment types if present
payment_counts = payment_counts[payment_counts['payment_type'] != 'not defined']

# Step 4: Create Interactive Donut Chart
base = alt.Chart(payment_counts).encode(
    theta=alt.Theta("count", stack=True)
).properties(
    title='Payment Method Distribution',
    width=400,
    height=400
)

# Donut chart with interactive tooltips
donut_chart = base.mark_arc(outerRadius=120, innerRadius=80).encode(
    color=alt.Color("payment_type", title="Payment Method"),
    order=alt.Order("percentage", sort="descending"),
    tooltip=[
        alt.Tooltip("payment_type", title="Payment Method"),
        alt.Tooltip("count", title="Number of Transactions", format=","),
        alt.Tooltip("percentage", title="Percentage (%)", format=".1f")
    ]
)

donut_chart.show()

alt.Chart(...)

### Sales Trend Analysis

In [7]:
# Step 1: Data Preparation
# Convert purchase timestamp to datetime
orders['order_purchase_timestamp'] = pd.to_datetime(
    orders['order_purchase_timestamp'], 
    errors='coerce'
)

# Step 2: Merge orders with items
# Join orders with items to get price data
df_sales = pd.merge(orders, items, on='order_id', how='inner')

# Step 3: Filter for delivered orders only
# Only analyze completed transactions
df_sales = df_sales[df_sales['order_status'] == 'delivered'].copy()

# Step 4: Extract month-year for grouping
# Create a month-year column for time series aggregation
df_sales['month_year'] = df_sales['order_purchase_timestamp'].dt.strftime('%Y-%m')

# Step 5: Calculate monthly revenue
# Sum up all prices per month
monthly_revenue = df_sales.groupby('month_year')['price'].sum().reset_index()
monthly_revenue.columns = ['month_year', 'total_revenue']

# Step 6: Calculate Month-over-Month (MoM) growth
# Calculate percentage change from previous month
monthly_revenue['mom_growth'] = monthly_revenue['total_revenue'].pct_change() * 100

# Step 7: Identify Black Friday months (November)
# Extract month from month_year string
monthly_revenue['month'] = pd.to_datetime(monthly_revenue['month_year']).dt.month
monthly_revenue['is_black_friday'] = monthly_revenue['month'] == 11

itables.show(monthly_revenue)


Loading ITables v2.5.2 from the internet... (need help?)


In [8]:
# Step 8: Create Interactive Line Chart
fig = go.Figure()

# Add main revenue line
fig.add_trace(go.Scatter(
    x=monthly_revenue['month_year'],
    y=monthly_revenue['total_revenue'],
    mode='lines+markers',
    name='Monthly Revenue',
    line=dict(color='#636EFA', width=3),
    marker=dict(size=6),
    hovertemplate='<b>%{x}</b><br>' +
                  'Revenue: R$ %{y:,.2f}<br>' +
                  '<extra></extra>'
))

# Highlight Black Friday months
black_friday_data = monthly_revenue[monthly_revenue['is_black_friday']]
fig.add_trace(go.Scatter(
    x=black_friday_data['month_year'],
    y=black_friday_data['total_revenue'],
    mode='markers',
    name='Black Friday (November)',
    marker=dict(
        size=15,
        color='#EF553B',
        symbol='star',
        line=dict(color='white', width=2)
    ),
    hovertemplate='<b>BLACK FRIDAY</b><br>' +
                  '%{x}<br>' +
                  'Revenue: R$ %{y:,.2f}<br>' +
                  '<extra></extra>'
))

# Update layout
fig.update_layout(
    title={
        'text': 'Monthly Revenue Trend (2016-2018)<br><sub>Red stars indicate Black Friday months (November)</sub>',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Month-Year',
    yaxis_title='Total Revenue (R$)',
    hovermode='x unified',
    height=600,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Format y-axis to show currency
fig.update_yaxes(tickformat=',.0f')

fig.show()


#### Key Insights
 

In [9]:
# Step 9: Key Insights and Analysis

# Overall growth metrics
first_month = monthly_revenue.iloc[0]
last_month = monthly_revenue.iloc[-1]
total_growth = ((last_month['total_revenue'] - first_month['total_revenue']) / 
                first_month['total_revenue']) * 100

print("\nOVERALL GROWTH:")
print("-" * 80)
print(f"Period: {first_month['month_year']} to {last_month['month_year']}")
print(f"Starting Revenue: R$ {first_month['total_revenue']:,.2f}")
print(f"Ending Revenue: R$ {last_month['total_revenue']:,.2f}")
print(f"Total Growth: {total_growth:,.1f}%")

# Peak and lowest months
peak_month = monthly_revenue.loc[monthly_revenue['total_revenue'].idxmax()]
lowest_month = monthly_revenue.loc[monthly_revenue['total_revenue'].idxmin()]

print("\nPEAK PERFORMANCE:")
print("-" * 80)
print(f"Best Month: {peak_month['month_year']}")
print(f"Revenue: R$ {peak_month['total_revenue']:,.2f}")
if peak_month['is_black_friday']:
    print("Note: This is a Black Friday month (November)")

print("\nLOWEST PERFORMANCE:")
print("-" * 80)
print(f"Worst Month: {lowest_month['month_year']}")
print(f"Revenue: R$ {lowest_month['total_revenue']:,.2f}")

# Black Friday impact analysis
bf_months = monthly_revenue[monthly_revenue['is_black_friday']]
non_bf_avg = monthly_revenue[~monthly_revenue['is_black_friday']]['total_revenue'].mean()
bf_avg = bf_months['total_revenue'].mean()
bf_impact = ((bf_avg - non_bf_avg) / non_bf_avg) * 100

print("\nBLACK FRIDAY IMPACT:")
print("-" * 80)
print(f"Average Revenue (Non-Black Friday): R$ {non_bf_avg:,.2f}")
print(f"Average Revenue (Black Friday): R$ {bf_avg:,.2f}")
print(f"Black Friday Boost: {bf_impact:,.1f}%")
print("\nBlack Friday Months:")
for _, row in bf_months.iterrows():
    print(f"  - {row['month_year']}: R$ {row['total_revenue']:,.2f}")

# Growth trend analysis
# Calculate average MoM growth (excluding first month which has NaN)
avg_mom_growth = monthly_revenue['mom_growth'].dropna().mean()

print("\nGROWTH TREND:")
print("-" * 80)
print(f"Average Month-over-Month Growth: {avg_mom_growth:,.1f}%")

if avg_mom_growth > 5:
    print("Status: STRONG GROWTH - Business is expanding rapidly")
elif avg_mom_growth > 0:
    print("Status: POSITIVE GROWTH - Business is growing steadily")
elif avg_mom_growth > -5:
    print("Status: STABLE - Revenue is relatively flat")
else:
    print("Status: DECLINING - Business needs attention")

# Best and worst growth months
best_growth = monthly_revenue.loc[monthly_revenue['mom_growth'].idxmax()]
worst_growth = monthly_revenue.loc[monthly_revenue['mom_growth'].idxmin()]

print("\nBEST GROWTH MONTH:")
print(f"  {best_growth['month_year']}: +{best_growth['mom_growth']:.1f}% MoM")

print("\nWORST GROWTH MONTH:")
print(f"  {worst_growth['month_year']}: {worst_growth['mom_growth']:.1f}% MoM")

# Seasonality check
print("\nSEASONALITY PATTERNS:")
print("-" * 80)
seasonal_data = monthly_revenue.copy()
seasonal_data['month_name'] = pd.to_datetime(seasonal_data['month_year']).dt.strftime('%B')
seasonal_avg = seasonal_data.groupby('month')['total_revenue'].mean().sort_values(ascending=False)

print("Top 3 Months by Average Revenue:")
for i, (month_num, revenue) in enumerate(seasonal_avg.head(3).items(), 1):
    month_name = pd.to_datetime(f'2020-{month_num:02d}-01').strftime('%B')
    print(f"  {i}. {month_name}: R$ {revenue:,.2f}")

print("=" * 80)


OVERALL GROWTH:
--------------------------------------------------------------------------------
Period: 2016-09 to 2018-08
Starting Revenue: R$ 134.97
Ending Revenue: R$ 838,576.64
Total Growth: 621,205.9%

PEAK PERFORMANCE:
--------------------------------------------------------------------------------
Best Month: 2017-11
Revenue: R$ 987,765.37
Note: This is a Black Friday month (November)

LOWEST PERFORMANCE:
--------------------------------------------------------------------------------
Worst Month: 2016-12
Revenue: R$ 10.90

BLACK FRIDAY IMPACT:
--------------------------------------------------------------------------------
Average Revenue (Non-Black Friday): R$ 556,078.76
Average Revenue (Black Friday): R$ 987,765.37
Black Friday Boost: 77.6%

Black Friday Months:
  - 2017-11: R$ 987,765.37

GROWTH TREND:
--------------------------------------------------------------------------------
Average Month-over-Month Growth: 47,978.6%
Status: STRONG GROWTH - Business is expanding rap

### Correlation Study 

In [10]:
order_stats = items.groupby('order_id')[['price', 'freight_value']].sum().reset_index()
itables.show(order_stats)

Loading ITables v2.5.2 from the internet... (need help?)


In [ ]:

# Merge items with products to get weight and dimensions
items_prod = pd.merge(items, products, on='product_id', how='left')
items_prod['volume_cm3'] = items_prod['product_length_cm'] * items_prod['product_height_cm'] * items_prod['product_width_cm']

# Select columns for correlation
corr_cols = ['price', 'freight_value', 'product_weight_g', 'volume_cm3']
df_corr = items_prod[corr_cols].dropna()

# Calculate Correlation Matrix ---
corr_matrix = df_corr.corr()

# Create Heatmap ---
x = list(corr_matrix.columns)
y = list(corr_matrix.index)
z = corr_matrix.values

fig = ff.create_annotated_heatmap(
    z,
    x=x,
    y=y,
    annotation_text=np.around(z, decimals=2),
    hoverinfo='z',
    colorscale='RdBu',
    zmin=-1,
    zmax=1
)

fig.update_layout(title_text='Correlation Heatmap: Price, Freight, Weight & Volume')
fig.show()

#### Key Insights
- Weight and volume are strongly correlated.
- Freight cost is linked to both weight and volume.
- Price shows weak correlation with weight and volume.
- Freight cost does not reflect product value


### GEOGRAPHICAL ANALYSIS

In [12]:
# ==============================================================================
# Question: Which states have the highest average freight cost?
# ==============================================================================

# Step 1: Prepare the data
# Merge orders with customers to get state information
orders_customers = pd.merge(
    orders_clean, 
    customers[['customer_id', 'customer_state']], 
    on='customer_id', 
    how='left'
)

# Merge with items to get freight values
geo_data = pd.merge(
    orders_customers,
    items[['order_id', 'freight_value']],
    on='order_id',
    how='left'
)

# Step 2: Calculate delivery time in days
# Formula: order_delivered_customer_date - order_purchase_timestamp
geo_data['delivery_days'] = (
    geo_data['order_delivered_customer_date'] - 
    geo_data['order_purchase_timestamp']
).dt.days

# Step 3: Group by state and calculate metrics
state_metrics = geo_data.groupby('customer_state').agg({
    'freight_value': 'mean',
    'delivery_days': 'mean',
    'order_id': 'count'  # Number of orders per state
}).reset_index()

# Rename columns for clarity
state_metrics.columns = ['state', 'avg_freight', 'avg_delivery_days', 'order_count']

# Round values for better readability
state_metrics['avg_freight'] = state_metrics['avg_freight'].round(2)
state_metrics['avg_delivery_days'] = state_metrics['avg_delivery_days'].round(1)

# Sort by average freight cost (descending)
state_metrics = state_metrics.sort_values('avg_freight', ascending=False)

# Step 4: Create Interactive Dual-Axis Chart using Plotly
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar chart for Average Freight Cost
fig.add_trace(
    go.Bar(
        x=state_metrics['state'],
        y=state_metrics['avg_freight'],
        name='Avg Freight Cost (R$)',
        marker_color='#636EFA',
        hovertemplate='<b>%{x}</b><br>' +
                      'Avg Freight: R$ %{y:.2f}<br>' +
                      '<extra></extra>'
    ),
    secondary_y=False
)

# Add line chart for Average Delivery Days
fig.add_trace(
    go.Scatter(
        x=state_metrics['state'],
        y=state_metrics['avg_delivery_days'],
        name='Avg Delivery Days',
        mode='lines+markers',
        line=dict(color='#EF553B', width=3),
        marker=dict(size=8),
        hovertemplate='<b>%{x}</b><br>' +
                      'Avg Delivery: %{y:.1f} days<br>' +
                      '<extra></extra>'
    ),
    secondary_y=True
)

# Update layout
fig.update_layout(
    title={
        'text': 'Freight Cost vs Delivery Time by State<br><sub>Sorted by Average Freight Cost (Highest to Lowest)</sub>',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='State',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=600
)

# Update y-axes titles
fig.update_yaxes(title_text="<b>Average Freight Cost (R$)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Average Delivery Days</b>", secondary_y=True)

fig.show()

#### Key Insights

In [13]:
# Step 5: Key Insights and Analysis

# Top 5 states with highest freight costs
print("\n TOP 5 STATES WITH HIGHEST FREIGHT COSTS:")
print("-" * 80)
top_freight = state_metrics.nlargest(5, 'avg_freight')[['state', 'avg_freight', 'avg_delivery_days', 'order_count']]
print(top_freight.to_string(index=False))

# Top 5 states with longest delivery times
print("\n TOP 5 STATES WITH LONGEST DELIVERY TIMES:")
print("-" * 80)
top_delivery = state_metrics.nlargest(5, 'avg_delivery_days')[['state', 'avg_delivery_days', 'avg_freight', 'order_count']]
print(top_delivery.to_string(index=False))

# States with high freight but low delivery time (efficient logistics)
print("\n EFFICIENT STATES (High Freight + Fast Delivery):")
print("-" * 80)
efficient = state_metrics[
    (state_metrics['avg_freight'] > state_metrics['avg_freight'].median()) &
    (state_metrics['avg_delivery_days'] < state_metrics['avg_delivery_days'].median())
][['state', 'avg_freight', 'avg_delivery_days', 'order_count']]
print(efficient.to_string(index=False))

# Correlation between freight and delivery time
correlation = state_metrics[['avg_freight', 'avg_delivery_days']].corr().iloc[0, 1]
print(f"\n CORRELATION (Freight vs Delivery Days): {correlation:.3f}")

if correlation > 0.5:
    print("   → Strong positive correlation: Higher freight = Longer delivery")
elif correlation > 0.3:
    print("   → Moderate positive correlation: Some relationship exists")
else:
    print("   → Weak correlation: Freight cost doesn't strongly predict delivery time")



 TOP 5 STATES WITH HIGHEST FREIGHT COSTS:
--------------------------------------------------------------------------------
state  avg_freight  avg_delivery_days  order_count
   PB        43.09               20.1          586
   RR        43.09               27.8           46
   RO        41.33               19.3          273
   AC        40.05               20.3           91
   PI        39.12               18.9          523

 TOP 5 STATES WITH LONGEST DELIVERY TIMES:
--------------------------------------------------------------------------------
state  avg_delivery_days  avg_freight  order_count
   RR               27.8        43.09           46
   AP               27.8        34.16           81
   AM               26.0        33.31          163
   AL               24.0        35.87          427
   PA               23.3        35.63         1054

 EFFICIENT STATES (High Freight + Fast Delivery):
--------------------------------------------------------------------------------
state  

### Pareto Principle (80/20 Rule)

In [14]:
# ==============================================================================
# Question: Do a small percentage of products drive the majority of revenue?
# ==============================================================================

# 1. Calculate revenue per product
product_revenue = df_sales.groupby('product_id')['price'].sum().reset_index()
product_revenue = product_revenue.sort_values('price', ascending=False)

# 2. Calculate cumulative calculations
product_revenue['cumulative_revenue'] = product_revenue['price'].cumsum()
total_revenue = product_revenue['price'].sum()
product_revenue['cumulative_percentage'] = (product_revenue['cumulative_revenue'] / total_revenue) * 100

# 3. Add product rank (percentage of products)
product_revenue['rank'] = range(1, len(product_revenue) + 1)
product_revenue['product_pct'] = (product_revenue['rank'] / len(product_revenue)) * 100

# 4. Create Lorenz Curve
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=product_revenue['product_pct'],
    y=product_revenue['cumulative_percentage'],
    mode='lines',
    name='Cumulative Revenue',
    line=dict(color='#636EFA', width=3)
))

# Add 80/20 reference lines
fig.add_shape(type="line",
    x0=0, y0=80, x1=100, y1=80,
    line=dict(color="gray", dash="dash")
)
fig.add_shape(type="line",
    x0=20, y0=0, x1=20, y1=100,
    line=dict(color="gray", dash="dash")
)

fig.update_layout(
    title='Pareto Analysis (Lorenz Curve)',
    xaxis_title='% of Products',
    yaxis_title='% of Revenue',
    height=600,
    showlegend=False
)

fig.show()

#### Key Insights

- Top 20% of products generate 74.8% of total revenue


### Delivery Accuracy Histogram

In [15]:

# Step 1: Calculate the difference between estimated and actual delivery dates
# Positive values = Early delivery, Negative values = Late delivery, Zero = On time
orders_clean['delivery_diff'] = (
    orders_clean['order_estimated_delivery_date'] - 
    orders_clean['order_delivered_customer_date']
).dt.days

# Step 2: Calculate key statistics
late_deliveries = (orders_clean['delivery_diff'] < 0).mean() * 100
early_deliveries = (orders_clean['delivery_diff'] >= 0).mean() * 100
avg_diff = orders_clean['delivery_diff'].mean()

# Step 3: Create histogram visualization
fig = px.histogram(
    orders_clean,
    x='delivery_diff',
    nbins=60,  # Number of bins for better distribution
    title='Delivery Accuracy: Estimated vs Actual',
    labels={'delivery_diff': 'Days Difference '},
    color_discrete_sequence=['#00CC96']  # Green color for bars
)

# Step 4: Add reference lines
# Red dashed line at 0 (On time deliveries)
fig.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="On Time")

# Blue dotted line at average difference
fig.add_vline(x=avg_diff, line_dash="dot", line_color="blue",
              annotation_text=f"Avg Diff = {avg_diff:.1f} days")

# Step 5: Add annotation with summary statistics
fig.add_annotation(
    x=avg_diff, y=orders_clean['delivery_diff'].value_counts().max(),
    text=f"Late: {late_deliveries:.1f}% | Early/On-time: {early_deliveries:.1f}%",
    showarrow=False, font=dict(color="black", size=12), bgcolor="white"
)

# Step 6: Update layout for better readability
fig.update_layout(
    xaxis_title='Days Difference (Positive = Early, Negative = Late)',
    yaxis_title='Number of Orders',
    bargap=0.05,
    template="plotly_white"
)

# Step 7: Show the figure
fig.show()

#### Key Insights

- Percentage of Late Deliveries     : 8.1%
- Percentage of Early/On-time Orders: 91.9%
- Average Difference (days)         : 10.9

### Peak Activity Patterns

In [16]:
# ==============================================================================
# Question: When do customers shop the most?
# ==============================================================================

# 1. Extract Hour and Day of Week
orders_clean['hour'] = orders_clean['order_purchase_timestamp'].dt.hour
orders_clean['day_of_week'] = orders_clean['order_purchase_timestamp'].dt.day_name()

# Order days correctly
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
orders_clean['day_of_week'] = pd.Categorical(orders_clean['day_of_week'], categories=days_order, ordered=True)

# 2. Pivot Table for Heatmap
heatmap_data = orders_clean.groupby(['day_of_week', 'hour']).size().reset_index(name='count')
heatmap_pivot = heatmap_data.pivot(index='day_of_week', columns='hour', values='count')

# 3. Create Heatmap
fig = px.imshow(
    heatmap_pivot,
    labels=dict(x="Hour of Day", y="Day of Week", color="Number of Orders"),
    x=heatmap_pivot.columns,
    y=heatmap_pivot.index,
    color_continuous_scale='Viridis',
    title='Peak Shopping Times Heatmap'
)

fig.update_layout(height=500)
fig.show()

C:\Users\w\AppData\Local\Temp\ipykernel_9152\6851993.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



#### Key Insights

- Weekdays show higher shopping activity than weekends.
- Midday to early evening is the busiest period.
- Sunday has the lowest overall order volume.


RFM Segmentation

In [17]:
df_customers = df_sales.merge(customers, on='customer_id')
itables.show(df_customers)

Loading ITables v2.5.2 from the internet... (need help?)


In [18]:
# ==============================================================================
# Task: Create Recency, Frequency, Monetary dataframe
# ==============================================================================

# 1. Set Reference Date (1 day after max purchase date)
max_date = df_customers['order_purchase_timestamp'].max() + pd.Timedelta(days=1)


# 2. Calculate RFM Metrics
rfm = df_customers.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (max_date - x.max()).days, # Recency
    'order_id': 'count',                                             # Frequency
    'price': 'sum'                                                   # Monetary
}).reset_index()

rfm.columns = ['customer_unique_id', 'Recency', 'Frequency', 'Monetary']

# 3. Visualize 3D Scatter Plot
fig = px.scatter_3d(
    rfm.sample(n=5000, random_state=42), 
    x='Recency', 
    y='Frequency', 
    z='Monetary',
    color='Monetary',
    opacity=0.7,
    title='3D View of RFM Segments '
)

fig.update_layout(height=700)
fig.show()


In [19]:
high_monetary = rfm[rfm['Monetary'] > rfm['Monetary'].quantile(0.90)]

itables.show(high_monetary)

Loading ITables v2.5.2 from the internet... (need help?)


#### Key Insights

- While some customers with high total spending—such as the top spender who purchased eight times—do show repeated activity, the overall pattern in the 3D scatter plot suggests that most high-value customers tend to purchase infrequently and haven't returned in a long time. This indicates that high monetary value is often concentrated among customers with low frequency and high recency.